<a href="https://colab.research.google.com/github/haleemaraza/agent/blob/main/agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import asyncio

from agents import Agent, Runner, function_tool, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from google.colab import userdata

set_tracing_disabled(disabled=True)

MODEL='gemini/gemini-2.0-flash'
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

@function_tool
def get_tour_guide(city: str)->str:
  print(f"\n\n[DEBUG] Tour guide for {city}\n\n")
  return f"\n\n[DEBUG] Tour guide for {city}\n\n"

tour_guide_agent = Agent(
    name="Tour guide Assistant",
    instructions="you will answer tour guide relevant questions",
    model=LitellmModel(model=MODEL, api_key=GOOGLE_API_KEY),
    tools=[get_tour_guide],
    handoff_description="Tour guide assistant is specialized for all guide Queries"
)

food_guide_agent = Agent(
    name="Food guide Assistant",
    instructions="you will answer food relevant questions",
    model=LitellmModel(model=MODEL, api_key=GOOGLE_API_KEY),
    handoff_description= "Palate guide assistant is specialize for all food queries"
    # handoffs=[tour_guide_agent]
)

triage_agent= Agent(
    name="General assistant",
    instructions="you will chat with user for general questions and handoff to specilize agents you will always handoff tour_guide_agent and its tool too and food_guide_agent ",
    model=LitellmModel(model=MODEL, api_key=GOOGLE_API_KEY),
    handoffs=[food_guide_agent,tour_guide_agent]
)

result = Runner.run_sync(triage_agent, "i am going to the Paris what should i visit  first and eat")
print(result.final_output)
print ("-" * 10)
print(result.last_agent.name)